In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from point_e.util.plotting import plot_point_cloud
from point_e.util.point_cloud import PointCloud, PointNet, PARTNET_MODELS


In [3]:
n = 1
output_dir = "test_segmentation"
pointnet_models_dir = "/scratch/noam/pepsi/pointnet"
shapenet_dir = "/scratch/noam/shapetalk/point_clouds/scaled_to_align_rendering"
shapetalk_csv_path = "/scratch/noam/shapetalk/language/shapetalk_preprocessed_public_version_0.csv"

In [ ]:
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv(shapetalk_csv_path)
for shape_category in PARTNET_MODELS:
    shape_df = df[df["source_object_class"] == shape_category].sample(n)
    net = PointNet(shape_category, pointnet_models_dir)
    for _, row in shape_df.iterrows():
        uid = row.source_uid
        pc = PointCloud.load_shapenet(
            shapenet_dir,
            uid,
            shape_category,
        )
        pc = pc.segment(net)
        plot_point_cloud(pc).savefig(f"{output_dir}/{uid.replace('/', '_')}.png")
        plt.close()
        pc.render(f"{output_dir}/{uid.replace('/', '_')}_render")
        for part in PARTNET_MODELS[shape_category].parts:
            part_pc = pc.mask(part)
            plot_point_cloud(part_pc).savefig(
                f"{output_dir}/{uid.replace('/', '_')}_part_{part}.png"
            )
            plt.close()
            part_pc.render(f"{output_dir}/{uid.replace('/', '_')}_part_{part}_render")
        for file in os.listdir(output_dir):
            if file.endswith(".xml"):
                os.remove(f"{output_dir}/{file}")